In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

In [2]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


model = NeuralNetwork()

In [4]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [5]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # 예측(prediction)과 손실(loss) 계산
        pred = model(X)
        loss = loss_fn(pred, y)

        # 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100 * correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t + 1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.293610  [    0/60000]
loss: 2.282732  [ 6400/60000]
loss: 2.265985  [12800/60000]
loss: 2.265134  [19200/60000]
loss: 2.246273  [25600/60000]
loss: 2.217985  [32000/60000]
loss: 2.226525  [38400/60000]
loss: 2.191534  [44800/60000]
loss: 2.196080  [51200/60000]
loss: 2.164135  [57600/60000]
Test Error: 
 Accuracy: 47.0%, Avg loss: 2.155101 

Epoch 2
-------------------------------
loss: 2.165632  [    0/60000]
loss: 2.151520  [ 6400/60000]
loss: 2.095112  [12800/60000]
loss: 2.113142  [19200/60000]
loss: 2.055319  [25600/60000]
loss: 2.001009  [32000/60000]
loss: 2.024864  [38400/60000]
loss: 1.947029  [44800/60000]
loss: 1.961463  [51200/60000]
loss: 1.878344  [57600/60000]
Test Error: 
 Accuracy: 61.3%, Avg loss: 1.877416 

Epoch 3
-------------------------------
loss: 1.912188  [    0/60000]
loss: 1.875983  [ 6400/60000]
loss: 1.760412  [12800/60000]
loss: 1.802528  [19200/60000]
loss: 1.682747  [25600/60000]
loss: 1.640601  [32000/600